# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [73]:
%tensorflow_version 1.x

In [94]:
# Imagen para estilo
!wget http://www.artnet.com/WebServices/images/ll0038AlldXGbJFgUNECfDrCWvaHBOc0ppF/pierre-auguste-renoir-le-p%C3%AAcheur-%C3%A0-la-ligne.jpg

# Imagen para contenido
!wget https://cdn.flightsim.to/images/01/bNcnsf1e.png

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2020-11-12 17:43:32--  http://www.artnet.com/WebServices/images/ll0038AlldXGbJFgUNECfDrCWvaHBOc0ppF/pierre-auguste-renoir-le-p%C3%AAcheur-%C3%A0-la-ligne.jpg
Resolving www.artnet.com (www.artnet.com)... 104.17.211.181, 104.17.212.181
Connecting to www.artnet.com (www.artnet.com)|104.17.211.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84669 (83K) [image/jpeg]
Saving to: ‘pierre-auguste-renoir-le-pêcheur-à-la-ligne.jpg’

pierre-auguste-reno 100%[===================>]  82.68K  --.-KB/s    in 0.01s   

2020-11-12 17:43:32 (6.59 MB/s) - ‘pierre-auguste-renoir-le-pêcheur-à-la-ligne.jpg’ saved [84669/84669]

--2020-11-12 17:43:32--  https://cdn.flightsim.to/images/01/bNcnsf1e.png
Resolving cdn.flightsim.to (cdn.flightsim.to)... 172.67.72.219, 104.26.3.201, 104.26.2.201, ...
Connecting to cdn.flightsim.to (cdn.flightsim.to)|172.67.72.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3993491 (3.8M) [image/png]
Saving to: ‘bNcnsf1e.pn

In [95]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [96]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/bNcnsf1e.png")
style_reference_image_path = Path("/content/pierre-auguste-renoir-le-pêcheur-à-la-ligne.jpg")
result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

total_variation_weight
Peso de la variaton loss. Esto determina la correlación entre el estilo y el contenido de la imagen creada.

style_weight
Peso de la loss de estilo. Beta de la ecuación del paper

content_weight
Peso de la loss del contenido. Alpha en el ecuación del paper. 

La CNN utiliza varias capas para extraer patrones en vez de una sola capa.

In [97]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 50

In [98]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

load_img 
Carga la imagen. Su path, colores RGB con target_size y su dimensión con nrows y ncols.

img_to_array
convierte la imagen en numpy array.

expand_dims 
Suma una dimensión extra.

preprocess_input 
Prepocesa la imagen con vgg19 pasando la imagen de RGB a BGR y extrae de los canales la medida que necesita con valores 103.939, 116.779, y 123.68

In [99]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

Acá hacemos el proceso inverso, pasando de BGR a RGB. Eliminando los signos de los valores y a 8 bits con np.clip

In [100]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [101]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [102]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [103]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [104]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

Es el producto punto de los feature maps, lo utilizamos para encontrar la correlación entre los diferentes filtros. La idea es poder identificar los estilos de la imagen, sus colores y texturas.

- ¿Por qué se permutan las dimensiones de x?

Para multiplicar la matriz y obtener filas por columnas.

In [105]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:
style_loss:
Sumatoria de las diferencias en la correlación entre los features maps de la imagen inicial y la generada. Mientras menor sea la loss mayor será la coincidencia de estilos entre ambas imagenes.

content_loss:
Mide la distancia entre los features entre ambas imagenes buscando que la creada sea lo más parecida a la orginal.

total_variation_loss:
Mide la coherencia de la imagen generada a nivel visual. 

In [106]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [107]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [108]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [109]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [110]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

La función fmin_l_bfgs_b se utiliza para optimizar la loss, utilizando la loss y el gradiente como imputs y calculandose al mismo tiempo, esto se logra con el Evaluator.  En la última celda se ve la loss y el gradiente en cada iteración.

La diferencia con el paper es que acá no estan utilizando back-propagation, más bien usan fmin_l_bfgs_b.

Una alternativa puede ser ADAM.

In [111]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [112]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [113]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 22369237000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 5s
Start of iteration 1
Current loss value: 12970832000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 5s
Start of iteration 2
Current loss value: 10214689000.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 5s
Start of iteration 3
Current loss value: 8931549000.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 5s
Start of iteration 4
Current loss value: 8214672000.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 5s
Start of iteration 5
Current loss value: 7735055000.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 5s
Start of iteration 6
Current loss value: 7402647000.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 5s
Start of iteration 7
Cur

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

total_variation_weight:
Mientras más elevada la Loss la imagen sale más borrosa.
style_weight:
Mientras más elevado la imagen output se asemeja a la imagen estilo.
content_weight:
Mientras más elevado la imagen output se asemeja a la original.

Todas en 0 simplemente no pasa nada, es decir, la imagen output es la misma que la orginal.


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: adjunto una carpeta.